In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
absa = yf.Ticker("ABG.JO")
stock_info = absa.info

In [3]:
fundamentals = ['symbol', 'shortName', 'sector', 'marketCap', 'trailingEps', 'trailingPE', 'returnOnEquity', 'priceToBook', 'payoutRatio',
                'priceToSalesTrailing12Months', 'freeCashflow', 'dividendYield', 'debtToEquity', 'pegRatio']

In [4]:
for attr in fundamentals:
    print(f'{attr} \t\t\t {stock_info[attr]}')

symbol 			 ABG.JO
shortName 			 Absa Group Limited
sector 			 Financial Services
marketCap 			 13395030966272
trailingEps 			 16.349
trailingPE 			 987.15515
returnOnEquity 			 0.11218
priceToBook 			 110.10596
payoutRatio 			 0
priceToSalesTrailing12Months 			 186.48239
freeCashflow 			 None
dividendYield 			 0.019
debtToEquity 			 None
pegRatio 			 None


Import stock data

In [5]:
data = pd.read_csv('stock_list.csv')
# View first few rows
data.head()

,JSE_code,Company,Market_cap,Sector
0,4SI,4Sight Holdings Ltd.,1.385700e+08,Financials
1,ABG,Absa Group Ltd.,1.262050e+11,Financials
2,ABSP,Absa Bank Ltd.,3.642400e+09,Financials
3,ACL,ArcelorMittal South Africa Ltd.,1.001490e+10,Basic Materials
4,ACS,Acsion Ltd.,2.567240e+09,Financials


Let's get the list of stocks. We need to add ".JO" to each ticker to signify that the stock data we want is for stocks listed on the JSE.

In [6]:
stock_list = data.JSE_code.tolist()
# Append ".JO"
stock_list = [x + ".JO" for x in stock_list]
# Check outcome
stock_list[0:5]

['4SI.JO', 'ABG.JO', 'ABSP.JO', 'ACL.JO', 'ACS.JO']

We need to get the latest market cap and sector information for each stock. In addition, we will also get the following fundamentals data:
* Earnings per share, EPS - trailing
* Price to earnings, PE ratio - trailing 
* Return on equity, ROE 
* Price to book value, PB ratio
* Dividend payout ratio
* Price to sales value, PS ratio - trailing
* Free Cash Flow, FCF
* Divident yield
* Debt to equity ratio,
* Projected earnings growth rate, PEG

We will store this information in lists which we will use to create a DataFrame.

In [7]:
# Create empty lists
list_names = ['ticker', 'name', 'sector', 'market_cap', 'EPS', 'PE_ratio', 'ROE', 'PB_ratio', 
              'div_payout_ratio', 'PS_ratio', 'FCF', 'divident_yield', 'debt_equity', 'earnings_growth']

# Create a dictionary to store our lists
stock_fund = defaultdict(list)

Let's get the fundamentals data from Yahoo! Finance. This process will take a while...

In [8]:
# For every stock in our list.
for ticker in stock_list:
    # get stock data and store it in 'data'
    data = yf.Ticker(ticker)
    # we need the stock info, store it in 'info'
    info = data.info
    # let's get the fundamentals data and store it in a list with the same name
    for lst, attr in zip(list_names, fundamentals):
        try:
            stock_fund[lst].append(info[attr])  
        except KeyError: 
            stock_fund[lst].append(np.nan)

print('Processing done')

Processing done


Let's check the length of each of list

In [11]:
for name, array in stock_fund.items():
    print(f'{name} \t {len(array)}')


ticker 	 348
name 	 348
sector 	 348
market_cap 	 348
EPS 	 348
PE_ratio 	 348
ROE 	 348
PB_ratio 	 348
div_payout_ratio 	 348
PS_ratio 	 348
FCF 	 348
divident_yield 	 348
debt_equity 	 348
earnings_growth 	 348


Now that we have all the stock info we need, we'll store it in a DataFrame. 

In [14]:
# Create DataFrame
df = pd.DataFrame(stock_fund)
# Check few rows
df.head()

,ticker,name,sector,market_cap,EPS,PE_ratio,ROE,PB_ratio,div_payout_ratio,PS_ratio,FCF,divident_yield,debt_equity,earnings_growth
0,4SI.JO,4Sight Holdings Ltd,Technology,1.581677e+10,-0.015,NaN,0.02316,51.72414,0.0,32.01226,1.271870e+08,NaN,4.158,NaN
1,ABG.JO,Absa Group Limited,Financial Services,1.339503e+13,16.349,987.15515,0.11218,110.10596,0.0,186.48239,NaN,0.0190,NaN,NaN
2,ABSP.JO,ABSA Bank Ltd Pref,Financial Services,3.631238e+11,17.941,4514.79830,0.09397,391.96520,NaN,7.510627,NaN,0.0606,NaN,NaN
3,ACL.JO,ArcelorMittal SA Limited,Basic Materials,1.058880e+12,2.326,408.42648,0.84142,230.07991,0.0,33.911274,1.944500e+09,NaN,148.718,NaN
4,ACS.JO,Acsion Limited,Real Estate,1.647768e+11,1.494,280.45517,0.08583,21.17124,0.0,225.91226,1.832291e+08,NaN,15.684,NaN


Let's save the data as a csv file `fundamentals.csv`. In the next stage of the process, we'll clean the data.

In [15]:
df.to_csv('fundamentals.csv', index=False)